In [52]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
# Cargar el archivo train.csv desde la misma carpeta
td = pd.read_csv('dataset_concatenado_con_desviacion.csv', low_memory=False)

# Ver las primeras filas del archivo
td.head()

,id,destination,route,alternate_route,departure_time,arrival_time,current_time,traffic_streets,distance_to_next_stop,time_estimate,...,incident_description,deviation,fecha,medio_transporte,linea,pasajeros,tiempo_espera_min,puntualidad,incidencias,desviacion
0,1.0,Puerta del Sol,Linea 3,Linea 9,19:49,20:00,19:58,Paseo del Prado,2.12,55.0,...,Tráfico denso,sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.789186
1,2.0,Plaza Castilla,Linea 9,Linea 7,02:23,02:36,02:50,Calle Atocha,2.44,21.0,...,Retraso por obras,sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.604428
2,3.0,Puerta del Sol,Linea 4,Linea 2,18:58,19:31,19:11,Calle de Bravo Murillo,0.60,29.0,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.053541
3,4.0,Retiro,Linea 1,Linea 8,13:40,14:00,13:43,Calle de Guzmán el Bueno,4.56,29.0,...,NaN,sí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.769928
4,5.0,Retiro,Linea 2,Linea 5,05:58,06:19,06:13,Calle Alcalá,3.57,48.0,...,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.175909


In [54]:
td.shape

(200000, 30)

In [55]:
td.columns

Index(['id', 'destination', 'route', 'alternate_route', 'departure_time',
       'arrival_time', 'current_time', 'traffic_streets',
       'distance_to_next_stop', 'time_estimate', 'day_of_week', 'time_of_day',
       'traffic_status', 'traffic_delay', 'vehicle_type',
       'alternate_route_available', 'alternate_route_time_estimate',
       'route_efficiency', 'incident_occurred', 'incident_type',
       'incident_description', 'deviation', 'fecha', 'medio_transporte',
       'linea', 'pasajeros', 'tiempo_espera_min', 'puntualidad', 'incidencias',
       'desviacion'],
      dtype='object')

In [56]:
td.drop(['id','desviacion','pasajeros'],axis = 1, inplace = True)
td.head()

,destination,route,alternate_route,departure_time,arrival_time,current_time,traffic_streets,distance_to_next_stop,time_estimate,day_of_week,...,incident_occurred,incident_type,incident_description,deviation,fecha,medio_transporte,linea,tiempo_espera_min,puntualidad,incidencias
0,Puerta del Sol,Linea 3,Linea 9,19:49,20:00,19:58,Paseo del Prado,2.12,55.0,domingo,...,sí,accidente,Tráfico denso,sí,NaN,NaN,NaN,NaN,NaN,NaN
1,Plaza Castilla,Linea 9,Linea 7,02:23,02:36,02:50,Calle Atocha,2.44,21.0,jueves,...,sí,avería,Retraso por obras,sí,NaN,NaN,NaN,NaN,NaN,NaN
2,Puerta del Sol,Linea 4,Linea 2,18:58,19:31,19:11,Calle de Bravo Murillo,0.60,29.0,domingo,...,no,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
3,Retiro,Linea 1,Linea 8,13:40,14:00,13:43,Calle de Guzmán el Bueno,4.56,29.0,lunes,...,no,NaN,NaN,sí,NaN,NaN,NaN,NaN,NaN,NaN
4,Retiro,Linea 2,Linea 5,05:58,06:19,06:13,Calle Alcalá,3.57,48.0,viernes,...,no,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
td.isnull().sum()

destination                      100000
route                            100000
alternate_route                  100000
departure_time                   100000
arrival_time                     100000
current_time                     100000
traffic_streets                  100000
distance_to_next_stop            100000
time_estimate                    100000
day_of_week                      100000
time_of_day                      100000
traffic_status                   100000
traffic_delay                    100000
vehicle_type                     100000
alternate_route_available        100000
alternate_route_time_estimate    150114
route_efficiency                 100000
incident_occurred                100000
incident_type                    149914
incident_description             149914
deviation                        100000
fecha                            100000
medio_transporte                 100000
linea                            100000
tiempo_espera_min                100000


***RELLENAR VALORES NULOS***

In [58]:

# Convertir las columnas a formato de fecha y hora
td['departure_time'] = pd.to_datetime(td['departure_time'])
td['arrival_time'] = pd.to_datetime(td['arrival_time'])

C:\Users\lisyj\AppData\Local\Temp\ipykernel_12096\2199002932.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  td['departure_time'] = pd.to_datetime(td['departure_time'])


In [43]:
# Calcular la diferencia entre arrival_time y departure_time
td['time_difference'] = (td['arrival_time'] - td['departure_time'])

In [44]:
# Rellenar current_time como el punto medio entre departure_time y arrival_time
td['current_time'] = td['departure_time'] + (td['time_difference'] / 2)


In [45]:
# Rellenar departure_time y arrival_time en función de current_time
td['departure_time'] = td['departure_time'].fillna(td['current_time'] - pd.Timedelta(minutes=30))
td['arrival_time'] = td['arrival_time'].fillna(td['current_time'] + pd.Timedelta(minutes=30))


In [46]:
# Eliminar valores NaN y duplicados usando set
destination_list = list(set(td['destination'].dropna()))

print(destination_list)


['Plaza Castilla', 'Gran Via', 'El Prado', 'Retiro', 'Puerta del Sol', 'Plaza Mayor', 'Madrid Rio', 'Chamartin', 'Santiago Bernabeu', 'Atocha']


In [47]:
# Eliminar valores NaN y duplicados usando set
destination_list = list(set(td['route'].dropna()))

print(destination_list)

['Linea 1', 'Linea 8', 'Linea 6', 'Linea 7', 'Linea 5', 'Linea 10', 'Linea 9', 'Linea 2', 'Linea 4', 'Linea 3']


In [48]:
# Lista de posibles destinos y rutas
destinations = ['Plaza Castilla', 'Gran Via', 'El Prado', 'Retiro', 'Puerta del Sol', 'Plaza Mayor', 'Madrid Rio', 'Chamartin', 'Santiago Bernabeu', 'Atocha']
routes = ['Linea 1', 'Linea 8', 'Linea 6', 'Linea 7', 'Linea 5', 'Linea 10', 'Linea 9', 'Linea 2', 'Linea 4', 'Linea 3']

# Rellenar 'destination' solo en los valores NaN con valores aleatorios
td.loc[td['destination'].isna(), 'destination'] = np.random.choice(destinations, size=td['destination'].isna().sum())

# Rellenar 'route' solo en los valores NaN con valores aleatorios
td.loc[td['route'].isna(), 'route'] = np.random.choice(routes, size=td['route'].isna().sum())


In [49]:
#rellenar valores nulos en una columna e funcion de otra
td['incidencias'] = td['incidencias'].fillna(td['deviation'])


In [50]:
td.isnull().sum()

destination                           0
route                                 0
alternate_route                  100000
departure_time                   100000
arrival_time                     100000
current_time                     100000
traffic_streets                  100000
distance_to_next_stop            100000
time_estimate                    100000
day_of_week                      100000
time_of_day                      100000
traffic_status                   100000
traffic_delay                    100000
vehicle_type                     100000
alternate_route_available        100000
alternate_route_time_estimate    150114
route_efficiency                 100000
incident_occurred                100000
incident_type                    149914
incident_description             149914
deviation                        100000
fecha                            100000
medio_transporte                 100000
linea                            100000
tiempo_espera_min                100000
